# Input Job by Job
Change this section every time  
Close PDF Reader APP to avoid error messages

## Job Description

In [35]:
no_need_to_form_pdf = False
# designed for job search websites where you can directly copy paste the cover letter without bolding keywords nor formatting a pdf

In [36]:
job_title = "Analyst, Operations"

jd = """
Country: Hong Kong
To work with Senior Manager to control over day-to-day operations ensuring that the Bank's policies, operational guidelines are adhered to by team members, consistent delivery of quality services to clients and optimal utilization of available resources.

Key Responsibilities
1. Review Import/Export documents presented against LC terms and input to relevant trade system.
2. Handle transactions of Discounting/ Forfeiting/ Import & Export Finance/ Factoring.
3. Ensure transactions meeting service standard and in accordance with procedure manual.
4. Conduct AML/ Sanction checking of transactions. Report any suspicious activities per policies and guidelines.
5. Liaise with internal/external stakeholders including cross countries working partners to resolve Trade related issues.
6. Perform ad hoc duties and projects assigned by Management as well as cross team support as required to Banking Operations.

Requirements
1. Bachelor degree in Business Administration, Operation Management, or related disciplines preferred with solid 3-5 year experience in Trade Services.
2. Good knowledge in ICC UCP & URC and ISBP.
3. Sound knowledge in operational risk framework and sensitive to identify irregularities.
4. Exposure in SWIFT Alliance and Banking Systems.
5. Proficient in both spoken and written English and Chinese (including Putonghua).
6. Hands-on experience in MS Words & Excel.
"""

## Company info  
You can leave the address blank, Google Map API will help you find the address.

In [37]:
company_name = "Santander"
company_address_line_1 = "10/F, Two International Finance Centre"
company_address_line_2 = "8 Finance Street, Central"
company_city = "Hong Kong"
company_state_two_letter = "HK"
company_country = "Hong Kong"  # US, Canada, HK

## Personal-specific Info Regarding This Role
Do you have anything else to stress in your cover letter?

In [38]:
additional_strength_to_mention = """
- Extensive experience in fund accounting, fund management, and financial analysis
- Strong knowledge of financial products, services, and investment strategies
- Proficient in Bloomberg, FactSet
- Hong Kong permanent resident
"""

In [39]:
if company_country == "Hong Kong":
    additional_strength_to_mention = (
        "- Stress my tie to Hong Kong (for example, my bachelor degree from University of Hong Kong, my Hong Kong permanent recidency, my trilingual, etc.), better stress it from the beginning, since Hong Kong HRs kind of oppose non-locals without a strong tie to Hong Kong. \n"
        + additional_strength_to_mention
    )

Which version of CV do you want to use?

In [40]:
# cv_type = "trader"
# cv_type = "trader_hk"
# cv_type = "operation"
# cv_type = "risk"
# cv_type = "pan_finance"
cv_type = "pan_finance_hk"

# The Automation

Do not change unless bugs appeared.

In [41]:
import os
import re
import shutil
import pyperclip
import pandas as pd

from config import *
from gpt_functions import *
from gmap_functions import *
from notion_functions import *

from docx2pdf import convert
from datetime import datetime
from PyPDF2 import PdfFileReader, PdfFileWriter

In [42]:
cv_to_use = cv_dict[cv_type]
cv_location = cv_folder + "/" + cv_location_dict[cv_type]

Chromedriver Location

In [43]:
path_to_chromedriver, service, options = ready_for_chromedriver(
    binary_location, path_to_chromedriver
)

In [44]:
jd = jd.replace("–", "-").replace("'", "'")

# Get Company Location Straight

In [45]:
prompt_companyinfo = query_companyinfo(jd)

In [46]:
info = chatgpt("gpt-4o-mini", prompt_companyinfo)

In [47]:
(
    valid_info,
    company_name,
    company_address_line_1,
    company_address_line_2,
    company_city,
    company_state_two_letter,
    company_country,
) = company_info(
    info,
    company_name,
    company_address_line_1,
    company_address_line_2,
    company_city,
    company_state_two_letter,
    company_country,
)

In [48]:
print(f"Company Name: {company_name}")
print(f"Company Address Line 1: {company_address_line_1}")
print(f"Company Address Line 2: {company_address_line_2}")
print(f"City: {company_city}")
print(f"State: {company_state_two_letter}")
print(f"Country: {company_country}")

Company Name: Santander
Company Address Line 1: 10/F, Two International Finance Centre
Company Address Line 2: 8 Finance Street, Central
City: Hong Kong
State: HK
Country: Hong Kong


Check Area if any bug

In [49]:
# info

In [50]:
# info_test = info.replace(r"\n\n", r"\n")
# pattern = r"\*\*Company Name:\*\*\s*(.*?)\s*\n\*\*Company Address Line 1:\*\*\s*(.*?)\s*\n\*\*Company Address Line 2:\*\*\s*(.*?)\s*\n\*\*City:\*\*\s*(.*?)\s*\n\*\*State:\*\*\s*([A-Za-z]{2,})\s*"
# results = re.match(pattern, info_test, re.DOTALL)
# results.groups()

In [51]:
can_continue, info_type = judge_info_type(
    company_name, company_address_line_1, company_address_line_2, company_city
)

# Google Map API

In [52]:
company_address_line_3 = ""

if info_type == "no_address":
    print("No address found, launching Google Maps search.")
    try:
        company_address_line_1, company_address_line_2, company_address_line_3 = (
            nearby_search_address(company_city, company_name, company_country)
        )
    except AssertionError:
        beep(system_used)
        company_address_line_1 = input(
            "No address found, please enter manually. Line 1: "
        )
        beep(system_used)
        company_address_line_2 = input("Please enter the address line 2: ")
        if any(
            x == "" for x in [company_city, company_state_two_letter, company_country]
        ):
            beep(system_used)
            company_address_line_3 = input(
                "Please enter City, State Abbr, Country as line 3:"
            )
        else:
            company_address_line_3 = (
                f"{company_city}, {company_state_two_letter}, {company_country}"
            )
if company_country == "Hong Kong":
    company_address_line_3 = "Hong Kong"

if company_address_line_3 == "":
    company_address_line_3 = (
        f"{company_city}, {company_state_two_letter}, {company_country}"
    )

# Prompts and ChatGPT API

## Step 1: Warn you if you applied before

In [53]:
df_path = f"{root}/Application History.xlsx"
if os.path.exists(df_path):
    df = pd.read_excel(df_path)
    beep_or_not, message, temp = compare_jd(jd, company_name, df, threshold=90)
    if len(temp) > 0:
        print(temp)

    if beep_or_not:
        stat = msgbox_similar_application(message, temp, system_used)
        if stat == "halt":
            raise SystemExit

## Step 2: Check if we need to combine pdf

In [54]:
query_sthimportant = query_something_important(jd)

In [55]:
answer = chatgpt("gpt-4o-mini", query_sthimportant)

showed_popup_1 = judge_something_important(answer, package_folder, system_used)
print(showed_popup_1)

False


## Step 3: Any inportant keywords in JD that I did not mention

In [56]:
extract_status, ats_score, missing_kw_list = extract_keywords(
    jd, cv_to_use, company_name, company_country
)

if not extract_status:
    assert False, "Keyword extraction failed. Please check the logs."

elif ats_score >= 70:
    print(f"ATS Score: {ats_score}, you will pass this round of screening.\n")
    print("Still, you may want to consider adding the following keywords:")
    print("\n".join(missing_kw_list))

else:
    rec = show_popup_keyword(system_used, missing_kw_list, ats_score)
    if rec == "Decline suggestion":
        print("Process cancelled.")
    elif rec == "Already changed":
        # shutil copy current cv to base.docx
        shutil.copy(
            cv_location.replace(".pdf", ".docx"),
            f"{os.path.dirname(cv_location)}/base.docx",
        )
        # Convert the docx file to pdf
        base_docx_path = f"{os.path.dirname(cv_location)}/base.docx"
        base_pdf_path = f"{os.path.dirname(cv_location)}/base.pdf"

        # Convert the docx file to pdf
        convert(base_docx_path, base_pdf_path)

        # Read the PDF and keep only the first page
        pdf_reader = PdfFileReader(base_pdf_path)
        pdf_writer = PdfFileWriter()
        pdf_writer.addPage(pdf_reader.getPage(0))

        cv_path = f"{os.path.dirname(cv_location)}/cv_with_keywords.pdf"
        with open(cv_path, "wb") as output_pdf:
            pdf_writer.write(output_pdf)

    else:
        assert False, "Invalid recommendation received."

Already Changed


  0%|          | 0/1 [00:00<?, ?it/s]

{'input': '/Users/fred/Library/CloudStorage/Dropbox/Documents/JHt/CV/hk/base.docx', 'output': '/Users/fred/Library/CloudStorage/Dropbox/Documents/JHt/CV/hk/base.pdf', 'result': 'error', 'error': 'Error: Message not understood.'}


SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Step 4: Draft Cover Letter

In [57]:
if company_name == "" or company_name == "None":
    assert False, "Company name is empty."

if additional_strength_to_mention.replace(" ", "") == "":
    additional_strength_to_mention = None

prompt_cl = query_cover_letter(
    one_sentence_bio,
    job_title,
    cv_to_use,
    jd,
    company_name,
    additional_strength_to_mention=additional_strength_to_mention,
)

In [58]:
answer = chatgpt("o1-preview", prompt_cl, temp=1)
match = re.search(
    r"Dear hiring manager,\s*(.*)\s*Sincerely,", answer, re.DOTALL | re.IGNORECASE
)

if match:
    extracted_text = (
        match.group(1).strip()
        if not no_need_to_form_pdf
        else match.group(1).strip().replace("**", "")
    )
    pyperclip.copy(extracted_text)
    print("The content has been copied to the clipboard.")
    print("")
    print(extracted_text)
else:
    print("The specified text was not found. Please do it manually. Original answer:")
    print(answer)
    raise SystemExit

The content has been copied to the clipboard.

As a Hong Kong permanent resident and a graduate of the University of Hong Kong, I am excited to apply for the Analyst, Operations position at Santander. My background in fund management and financial analysis, combined with fluency in English, Cantonese, and Mandarin, makes me well-suited for this role.

In my previous role as an Assistant Portfolio Manager at Privium Fund Management, I co-managed a US$200 million option selling strategy. This experience allowed me to apply option pricing techniques to optimize premium income and utilize asset pricing models to forecast asset prices. At Yong Rong Asset, I worked as a Junior Trader on a US$30 million high conviction sub-fund, where I analyzed trade flows and employed strategies like limit orders, VWAP, and TWAP to achieve optimal execution.

My proficiency with **SWIFT Alliance**, banking systems, and tools like Bloomberg and FactSet positions me to contribute effectively to your team. I h

In [59]:
extracted_text = shrink_text(extracted_text, threshold=300)

No need to shrink the text. The original text is already less than 300 words: 246 words.


In [60]:
word_count = len(extracted_text.split())
word_count

246

## Step 5: Get my best Title for this Job

In [61]:
prompt_job_title = query_job_title(jd, cv_to_use, extracted_text)

In [62]:
proposed_job_title = chatgpt("chatgpt-4o-latest", prompt_job_title)

if "*" not in proposed_job_title:
    proposed_job_title = "**" + proposed_job_title + "**"
print(proposed_job_title)

**Operations Analyst**


# Add Record

In [63]:
df_path = f"{root}/Application History.xlsx"
if not os.path.exists(df_path):
    df = pd.DataFrame(
        columns=[
            "Date",
            "Company_Name",
            "Position_Name",
            "CV_Used",
            "Company_City",
            "Company_State",
            "Cover_Letter",
            "Original_JD",
        ]
    )
    df.to_excel(df_path, index=False)

In [64]:
df = pd.read_excel(df_path, sheet_name="Sheet1")
first_sentence = extracted_text.split(".")[0]

today = datetime.today().strftime("%B %d, %Y")

row = {
    "Date": today,
    "Company Name": company_name,
    "Position Name": job_title,
    "Cover Letter": f"Cover Letter - {my_name}.pdf",
    "CV Used": cv_to_use,
    "Company City": company_city,
    "Original_JD": jd,
}

row_df = pd.DataFrame([row])
df = pd.concat([df, row_df], ignore_index=True)
df.fillna("", inplace=True)
df.drop_duplicates(subset=df.columns, keep="first", inplace=True)

df.to_excel(f"{root}/Application History.xlsx", index=False)

# Notion

In [65]:
if no_need_to_form_pdf:
    raise SystemExit

if re.search(r"_hk$", cv_type):
    paper_size = "a4"
else:
    paper_size = "letter"

In [66]:
replace_job_title_success, block_id = replace_simple(
    "TBA Title", proposed_job_title, page_id
)
print(replace_job_title_success)

Found block with text: TBA Title
Block 10ab1542-9002-80d4-abe7-ffca8866414d updated.
True


In [67]:
replace_date_success, block_id = replace_simple("TBA Date", today, page_id)
print(replace_date_success)

Found block with text: TBA Date
Block b87e1d4a-c6af-4e91-bf93-e928c0c44dbe updated.
True


In [68]:
if info_type == "single_line":
    parsed_address = (
        f"{company_address_line_1}\n{company_name}\n{company_address_line_3}"
    )
elif info_type == "no_address":
    parsed_address = (
        f"{company_address_line_1}\n{company_address_line_2}\n{company_address_line_3}"
    )

elif info_type == "full_address" and company_country != "Hong Kong":
    parsed_address = f"{company_address_line_1}\n{company_address_line_2}\n{company_city}, {company_state_two_letter}, {company_country}"

elif info_type == "full_address" and company_country == "Hong Kong":
    parsed_address = (
        f"{company_name}\n{company_address_line_1}\n{company_address_line_2}, Hong Kong"
    )

else:
    assert False, "Invalid info_type"

In [69]:
replace_address_success, block_id_address = replace_simple(
    "TBA Address", parsed_address, page_id, change_color=True, color_to="default"
)

Found block with text: TBA Address
Block 10bb1542-9002-8099-a9a2-eb53beff1332 updated with color 'default'.


In [70]:
replace_main_text_success, block_id_main = replace_simple(
    "TBA Main Text", extracted_text.strip(), page_id
)

Found block with text: TBA Main Text
Block 144b1542-9002-80df-bce5-e27adaa3e45f updated.


In [71]:
first_paragraph = extracted_text.split("\n")[0].replace("*", "").strip()
download_success = selenium_download_pdf(
    service, options, download_folder, website, first_paragraph, paper_size
)

Page is successful.
Page is successful, proceeding to download PDF.
PDF saved to ..//Cover Letter/Cover Letter - Fred Li.pdf


# Parse PDF

In [72]:
input_pdf_path = f"{download_folder}/Cover Letter - {my_name}.pdf"
remove_blank_pages_from_pdf(input_pdf_path)

Page 2 is blank. Removing...
Saved PDF with 1 non-blank pages.


# Copy CV, CL to Package Folder

In [73]:
cv_location

'..//CV/hk/pan_finance_hk.pdf'

In [74]:
shutil.copy(cv_location, f"{package_folder}/CV - {my_name}.pdf")
shutil.copy(
    f"{download_folder}/Cover Letter - {my_name}.pdf",
    f"{package_folder}/Cover Letter - {my_name}.pdf",
)

cl_backup_folder = f"{root}/Cover Letter Backup"
shutil.copy(
    f"{package_folder}/Cover Letter - {my_name}.pdf",
    f"{cl_backup_folder}/{today}_{company_name}_{company_city}.pdf",
)

'..//Cover Letter Backup/February 17, 2025_Santander_Hong Kong.pdf'

In [75]:
# combine all three PDFs in package folder

if showed_popup_1:
    beep(system_used)
    sequence = input(
        "Since GPT identified some modifications specefic to this JD, please enter the sequence of the PDFs to be combined, 1: CV; 2: CL; 3: Unofficial Transcript. e.g.: 123; 12; 13; 1, etc."
    )
else:
    sequence = "12"

merge_pdf(sequence, package_folder, my_name)

True

# Recover to Template for next use

In [76]:
replace_simple(proposed_job_title.replace("*", ""), "TBA Title", page_id)

Found block with text: Operations Analyst
Block 10ab1542-9002-80d4-abe7-ffca8866414d updated.


(True, '10ab1542-9002-80d4-abe7-ffca8866414d')

In [77]:
replace_simple(today, "TBA Date", page_id)

Found block with text: February 17, 2025
Block b87e1d4a-c6af-4e91-bf93-e928c0c44dbe updated.


(True, 'b87e1d4a-c6af-4e91-bf93-e928c0c44dbe')

In [79]:
replace_simple(company_address_line_1, "TBA Address", page_id)

Found block with text: Santander
10/F, Two International Finance Centre
8 Finance Street, Central, Hong Kong
Block 10bb1542-9002-8099-a9a2-eb53beff1332 updated.


(True, '10bb1542-9002-8099-a9a2-eb53beff1332')

In [80]:
blocks = get_page_blocks(page_id)

start_text = "Dear Hiring Manager,"
end_text = "Sincerely,"
replacement_text = "TBA Main Text"

replace_text_in_blocks(blocks, start_text, end_text, replacement_text)

Updated block 144b1542-9002-80df-bce5-e27adaa3e45f with new text.
Updated block 144b1542-9002-80df-bce5-e27adaa3e45f with new text.


True